In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [ ]:
df = pd.read_csv("dataset.csv")
df['Gender'] = df['Gender'].map({'F':0, 'M':1})
df['No-show'] = df['No-show'].map({'No':0,'Yes':1})
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])
df['wait_days'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days
df['wait_days'] = df['wait_days'].clip(lower=0)
df = df[df['Age'] >= 0]
min_age = df['Age'].min()
max_age = df['Age'].max()
df['Age'] = (df['Age'] - min_age)/(max_age-min_age)
min_wait = df['wait_days'].min()
max_wait = df['wait_days'].max()
df['wait_days'] = (df['wait_days'] - min_wait)/(max_wait-min_wait)
df = pd.get_dummies(df,columns=['Neighbourhood'],prefix='location',dtype=int)
delete_columns = ['PatientId','AppointmentID','ScheduledDay','AppointmentDay']
df = df.drop(columns=delete_columns)
column = 'No-show'
df = df[[column] + [col for col in df.columns if col != column]]

data = df.to_numpy()
np.random.shuffle(data)
data_train = data[:90000]
data_test = data[90000:]
x_train = data_train[:,1:].T
y_train = data_train[:,0].T
x_test = data_test[:,1:].T
y_test = data_test[:,0].T

In [ ]:
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def sigderiv(z):
    return (1-sigmoid(z))*sigmoid(z)

def relu(z):
    return np.maximum(z,0)

def relu_der(z):
    return (z > 0).astype(float)
def softmax(a):
    a_low = a - np.max(a,axis = 0,keepdims=True)
    p = np.exp(a_low)
    sum = np.sum(p,axis = 0,keepdims = True)
    return p / sum

size = [90,128,64,32,1]
layers = len(size)
def init_par():
    W = [np.random.randn(y,x)*np.sqrt(1/x) for x, y in zip(size[:-1],size[1:])]
    B = [np.zeros(y).reshape((-1,1)) for y in size[1:]]
    return W,B

def forward_prop(X,weight,bias):
    a = [X]
    z = []
    m = len(weight)
    for i,(W,B) in enumerate(zip(weight,bias)):
        z_next = W.dot(a[-1]) + B
        if(i != m-1):
            a_next = relu(z_next) 
        else:
            a_next = sigmoid(z_next)    
        a.append(a_next)
        z.append(z_next)
    return a, z

    
def back_prop(Y, a, z, W):
    m = Y.shape[0]
    l = len(z)
    dz = [None]*l
    db = [None]*l
    dw = [None]*l
    class_weight = (Y*1.6) + ((1-Y)*0.4)
    dz[l-1] = class_weight*(a[l] - Y)
    dw[l-1] = (1/m)*dz[l-1].dot(a[l-1].T)
    db[l-1] = (1/m)*np.sum(dz[l-1],axis=1,keepdims=True)
    j = l - 2
    while(j >= 0):
        dz[j] = relu_der(z[j])*(W[j+1].T.dot(dz[j+1])) 
        dw[j] = (1/m)*dz[j].dot(a[j].T)
        db[j] = (1/m)*np.sum(dz[j],axis=1,keepdims=True)
        j -= 1
    return dw,db


def update_para(w,b,dw,db,alpha):
    m = len(w)
    for i in range(m):
        w[i] = w[i] - (alpha*dw[i])
        b[i] = b[i] - (alpha*db[i]) 
    return w,b

def prediction(a,t = 0.5):
    predictions = (a[-1] >= t).astype(int)
    return predictions

def cost(a_last,y):
    ep = 1e-8
    a = np.clip(a_last,ep,1-ep)
    temp = -(1.6*y*np.log(a) + 0.4*(1-y)*np.log(1-a))
    cost = np.mean(temp)
    return cost

def accuracy(predictions,y):
    return np.mean(predictions == y)*100

def precision(predictions,y):
    TP = np.sum((predictions == 1) & (y == 1))
    FP = np.sum((predictions == 1) & (y == 0))
    if((TP+FP) > 0):
        return TP/(TP+FP)
    return 0

def recall(predictions,y):
    TP = np.sum((predictions == 1) & (y == 1))
    FN = np.sum((predictions == 0)&(y == 1))
    if(TP + FN > 0):
        return TP/(TP+FN)
    return 0 

def F1(p,r):
    if(p + r > 0):
        f1_score = 2*p*r/(p+r)
    else:
        f1_score = 0
    return f1_score
def confusion_matrix(y,p):
    y = y.reshape(-1)
    p = p.reshape(-1)
    confusion = np.zeros((2,2),dtype=int)
    for pre, ytrue in zip(p,y):
        confusion[int(ytrue),int(pre)] += 1
    return confusion

def create_batches(X,y,batch_size):
    m = X.shape[1]
    permutation = np.random.permutation(m)
    X_random = X[:,permutation]
    y_random = y[permutation]

    batches = []
    for k in range(0,m,batch_size):
        x_batch = X_random[:,k:k+batch_size]
        y_batch = y_random[k:k+batch_size]
        batches.append((x_batch,y_batch))
    return batches


In [ ]:
def gradient_descent(X,y,alpha,epochs, batch_size):
    w,b = init_par()
    costs = []
    for i in range(epochs):
        batches = create_batches(X,y,batch_size)
        for x_batch, y_batch in batches:
            a,z = forward_prop(x_batch,w,b)
            dw,db = back_prop(y_batch,a,z,w)
            w,b = update_para(w,b,dw,db,alpha)
        alpha = 0.999*alpha
        if((i+1) % 1 == 0):
            a_full,z_full = forward_prop(X,w,b)
            cost_total = cost(a_full[-1],y)
            costs.append(cost_total)
        if((i) % 50 == 0):
            a_full,z_full = forward_prop(X,w,b)
            cost_total = cost(a_full[-1],y)
            print("Epoch:", (i))
            print(f"cost: {cost_total:.2f}")
    
    a_full,z_full = forward_prop(X,w,b)
    fig, axs = plt.subplots(1,1)
    axs.plot(range(epochs),costs)
    axs.set_title("Cost vs Epochs")
    axs.set_ylabel("Cost")
    axs.set_xlabel("Epoch")
    axs.set_ylim(0,1)
    return w,b




In [ ]:
alpha = 0.02
epochs = 200
batch_size = 64
w,b = gradient_descent(x_train,y_train,alpha,epochs,batch_size)

In [ ]:
a,_ = forward_prop(x_train,w,b)
thresholds = np.linspace(0, 1, num=100)
precisions = []
recalls = []
for t in thresholds:
        pred = (a[-1] >= t).astype(int)
        p = precision(pred,y_train)
        r = recall(pred,y_train)
        precisions.append(p)
        recalls.append(r)
recalls = np.array(recalls)
precisions = np.array(precisions)
thresholds = np.array(thresholds)
with np.errstate(divide='ignore', invalid='ignore'):
    f1_scores = 2 * precisions * recalls / (precisions + recalls)
    f1_scores[np.isnan(f1_scores)] = 0
best_index = np.argmax(f1_scores)
best_f1 = f1_scores[best_index]
best_threshold = thresholds[best_index]
prec = precisions[best_index]
rec = recalls[best_index]
acc = accuracy(prediction(a,best_threshold),y_train)
print(f"Best F1 Score: {best_f1:.4f} at threshold: {best_threshold:.2f},\
      precision: {prec:.2f}, recall: {rec:.2f}, accuracy = {acc:.2f}")

sorted_index = np.argsort(recalls)
recalls = recalls[sorted_index]
precisions = precisions[sorted_index]
pr_auc = np.trapezoid(precisions, recalls)
print(f"PR-AUC: {pr_auc:.4f}")

conf = confusion_matrix(y_train,prediction(a,best_threshold))
fig, axs = plt.subplots(1,2)
im = axs[0].imshow(conf,cmap="Blues")
axs[0].set_title("Confusion Matrix")
axs[0].set_xlabel("Prediction")
axs[0].set_ylabel("True Value")
axs[0].set_xticks([0,1])
axs[0].set_yticks([0,1])
axs[0].set_xticklabels(['0','1'])
axs[0].set_yticklabels(['0','1'])
for i in range(2):
    for j in range(2):
        axs[0].text(j, i, conf[i, j], ha='center', va='center', fontsize=14, color='black')
axs[0].set_aspect('equal')

axs[1].plot(recalls, precisions, label="PR Curve", color='blue', linewidth=2)
axs[1].fill_between(recalls, precisions, alpha=0.2, color='blue')
axs[1].set_title("Precision-Recall Curve")
axs[1].set_xlabel("Recall")
axs[1].set_ylabel("Precision")
axs[1].set_xlim([0.0, 1.05])
axs[1].set_ylim([0.0, 1.05])
axs[1].grid(True)
axs[1].legend()

plt.tight_layout()
plt.show()



Now for testing data

In [ ]:
a,_ = forward_prop(x_test,w,b)
thresholds = np.linspace(0, 1, num=100)
precisions = []
recalls = []
for t in thresholds:
        pred = (a[-1] >= t).astype(int)
        p = precision(pred,y_test)
        r = recall(pred,y_test)
        precisions.append(p)
        recalls.append(r)
recalls = np.array(recalls)
precisions = np.array(precisions)
thresholds = np.array(thresholds)
with np.errstate(divide='ignore', invalid='ignore'):
    f1_scores = 2 * precisions * recalls / (precisions + recalls)
    f1_scores[np.isnan(f1_scores)] = 0
best_index = np.argmax(f1_scores)
best_f1 = f1_scores[best_index]
best_threshold = thresholds[best_index]
prec = precisions[best_index]
rec = recalls[best_index]
acc = accuracy(prediction(a,best_threshold),y_test)
print(f"Best F1 Score: {best_f1:.4f} at threshold: {best_threshold:.2f},\
      precision: {prec:.2f}, recall: {rec:.2f}, accuracy = {acc:.2f}")

sorted_index = np.argsort(recalls)
recalls = recalls[sorted_index]
precisions = precisions[sorted_index]
pr_auc = np.trapezoid(precisions, recalls)
print(f"PR-AUC: {pr_auc:.4f}")

conf = confusion_matrix(y_test,prediction(a,best_threshold))
fig, axs = plt.subplots(1,2)
im = axs[0].imshow(conf,cmap="Blues")
axs[0].set_title("Confusion Matrix")
axs[0].set_xlabel("Prediction")
axs[0].set_ylabel("True Value")
axs[0].set_xticks([0,1])
axs[0].set_yticks([0,1])
axs[0].set_xticklabels(['0','1'])
axs[0].set_yticklabels(['0','1'])
for i in range(2):
    for j in range(2):
        axs[0].text(j, i, conf[i, j], ha='center', va='center', fontsize=14, color='black')
axs[0].set_aspect('equal')

axs[1].plot(recalls, precisions, label="PR Curve", color='blue', linewidth=2)
axs[1].fill_between(recalls, precisions, alpha=0.2, color='blue')
axs[1].set_title("Precision-Recall Curve")
axs[1].set_xlabel("Recall")
axs[1].set_ylabel("Precision")
axs[1].set_xlim([0.0, 1.05])
axs[1].set_ylim([0.0, 1.05])
axs[1].grid(True)
axs[1].legend()

plt.tight_layout()
plt.show()
